In [ ]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [1]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
No ROCm runtime is found, using ROCM_HOME='/usr'
No CUDA runtime is found, using CUDA_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D pro

0it [00:00, ?it/s]

In [3]:
train_dataset = pd.read_excel('train_forms.xlsx')[['input_text','labels']]
train_dataset.drop_duplicates(['input_text']).reset_index(drop=True)
c = train_dataset['labels']=='other'
train_dataset.loc[c,'labels']=pd.NA
c = train_dataset['labels']=='birthdate'
train_dataset.loc[c,'labels']='birth'

train_dataset['labels'].unique()

#c = train_dataset['labels']=='privacy'
#train_dataset.loc[c,:]

lll=[]
person = {"first name":"majid","last name":"sharghi foroushani","pronoun":"he","birth":"10/08/1997","age":"27","sex":"male","skill":"NLP,LLM","country":"Germany","address":"Philip-9","city":"Erlangen","phone number":"+497773154","available from":"6/24/2024","salary":"10000","German language level":"A1","English language level":"C1","visa":"do not need visa","email":"sharghi.majid@gmail.com","password":"123456","find us":"Google","xing":"www.xing.com","job":"I am stduent","university":"FAU","major":"AI","privacy":"privacy","search":"search","submit":"submit","cookie":"cookie","cover letter":"cover letter","linkedin":"www.linkedin.com","github":"www.github.com","login":"login","register":"register","cv":"/pah/to/cv.pdf","twitter":"www.twitter.com"}
for k,v in person.items():
    c =  train_dataset['labels']==k
    #train_dataset.loc[c,'labels']=k
    pass
    
print(train_dataset['labels'].value_counts())


labels
privacy                   63
search                    43
email                     38
cv                        37
submit                    28
first name                25
phone number              25
cover letter              24
linkedin                  23
last name                 21
find us                   15
cancel                    12
address                   11
salary                    10
available from            10
github                     9
question                   8
transcript                 8
recommendation             5
login                      5
birth                      5
password                   4
german language level      4
xing                       4
next                       4
website                    3
company                    3
house number               3
zip                        3
picture                    3
cookie                     3
city                       3
register                   3
eligible                   2
usernam

In [96]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset['label'].unique()

array(['cv', 'first name', 'last name', 'email', 'phone number',
       'available from', 'github', 'privacy', 'submit', 'search',
       'country', 'sex', 'zip', 'cover letter', 'linkedin', 'visa',
       'xing', 'newsletter'], dtype=object)

In [5]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct'
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [6]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [7]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj'
                                                                                     #,'k_proj','o_proj'
                                                                                     ]))
model.print_trainable_parameters()
model

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=896, out_features=128, bias=True)
              (v_proj): lora.Linear(

"from sentence_transformers  import SentenceTransformer\n\nembeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir).to('cuda')\nembdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)\nembeding_model.eval()\nembeding_model"

In [9]:

c = test_dataset['label']=='guthub'
test_dataset.loc[c,'label'] = 'github'
test_dataset = test_dataset.reset_index(drop=True)

In [10]:
#test_dataset.to_excel('forms.xlsx',index=0)

In [11]:
x = test_dataset['query'] + test_dataset['element']
x = x.tolist()
x[0]

'Personal information\n CV or resume\n Browse CV to autocomplete your application. You can still fill your profile manually.\n <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>'

In [12]:
s='['
for i in LABEL_INDEX_TO_KEY[0]:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

'["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]'

In [13]:
"""def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset"""


'def f(x):\n    print(x)\n    return \'{"answer":"\'+ x +\'"}\'\n\ntrain_dataset[\'labels\'] = train_dataset[\'labels\'].map(f)\ntrain_dataset'

In [14]:
import random
def get_random():
    d = {}
    for k in person.keys():
        d[k]=str(random.randint(1000,10000))
        
    return d

#get_random()
def to_str(p):
    s='{'
    for k,v in p.items():
        s+=f'"{k}":"{v}",'
        pass

    s+='}'
    return s
    
    
def get_one_msg(text , label):
    p = get_random()
    strp = to_str(p)
    rc = ''#random.choice(list(p.keys()))+' '
    d = [#{"role":"system","content":""}
    { "role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+strp}
    ,{"role":"user", "content":"What is this HTML element about?\n"+rc+text }
    ,{"role":"assistant","content":'{"answer":"'+ p[label] +'"}'}
        ] 
    return d
def get_chat():

    d = []
    labels = []
    for i,x in train_dataset.iterrows():
        try:
                d.append(get_one_msg(x['input_text'],x['labels']))
        except:
            #print(x['labels'])
            pass
        
    return d#,labels

get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n{"first name":"9862","last name":"2132","pronoun":"4433","birth":"8365","age":"5812","sex":"3497","skill":"8661","country":"3614","address":"9927","city":"9491","phone number":"6231","available from":"2659","salary":"1620","German language level":"9856","English language level":"7687","visa":"4566","email":"6862","password":"1983","find us":"3543","xing":"4307","job":"2541","university":"4385","major":"6327","privacy":"8222","search":"5267","submit":"6613","cookie":"3401","cover letter":"5577","linkedin":"1510","github":"4420","login":"3472","register":"8307","cv":"1894","twitter":"3216",}'},
  {'role': 'user',
   'content': 'What is this HTML element about?\npersonal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type="file" tabindex="-1" style="display: n

In [15]:
test_dataset

,query,element,label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number
...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit


In [16]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+to_str(person)}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+str(x['query'])+x['element'] }
                               #,{"role":"assistant","content":""}
                               ] for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))

print(tokenizer.decode(token_ids=tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


414
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
{"first name":"8141","last name":"5649","pronoun":"8486","birth":"1436","age":"8301","sex":"2345","skill":"5028","country":"5945","address":"9450","city":"9895","phone number":"3560","available from":"5322","salary":"5197","German language level":"5245","English language level":"9102","visa":"4135","email":"9180","password":"8132","find us":"7543","xing":"4042","job":"1088","university":"2102","major":"1106","privacy":"6614","search":"1379","submit":"1802","cookie":"7749","cover letter":"2593","linkedin":"5636","github":"9853","login":"4421","register":"9471","cv":"3278","twitter":"6811",}<|im_end|>
<|im_start|>user
What is this HTML element about?
personal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type="file

In [17]:
tokenizer(['<|im_start|>assistant'])

{'input_ids': [[151644, 77091]], 'attention_mask': [[1, 1]]}

In [18]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y
        
mmf(no_label)['labels'].shape


torch.Size([100, 512])

In [25]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
        
"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass
        
    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()
            
        return self.dataset[i]
        pass
    
    def __len__(self):
        return self.len
        pass
    
    
    
        
        
trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=3,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,
          
          #eval_accumulation_steps =2,
          
            
        ))

trainer.train()

Step,Training Loss
300,0.129500
600,0.068600
900,0.119900
1200,0.066900


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=1242, training_loss=0.09981500468968194, metrics={'train_runtime': 155.7174, 'train_samples_per_second': 7.976, 'train_steps_per_second': 7.976, 'total_flos': 1001992680589824.0, 'train_loss': 0.09981500468968194, 'epoch': 3.0})

In [26]:
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o= model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break
            
            #print(x[0],x[1])
            try:
                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                c = answer==person[test_dataset.loc[i,'label']]
                
                if c:
                    acc +=1
                else:
                    print("NO_CORRECT:",answer,person[test_dataset.loc[i,'label']])
                    print("---------------------------------")
                    
                    pass
                    
                
                pass
            except Exception as e:
                print("ERROR: ",decoded)
                print("---------------------------------")
                
                print(e)
                
                pass
    
    acc/=len(no_label)
    print("acc:",acc)
    
    
    
compute_metric()

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


NO_CORRECT: majid sharghi foroushani
---------------------------------
NO_CORRECT: data-privacy-statements privacy
---------------------------------
NO_CORRECT: he male
---------------------------------
NO_CORRECT: majid sharghi foroushani
---------------------------------
ERROR:  <|im_start|>assistant
 {"answer":"Philip-9"}<|im_end|>
---------------------------------
'zip'
ERROR:  option><option value="country_belarus"> Belarus</option><option value="country_berlin">
---------------------------------
Expecting value: line 1 column 1 (char 0)
NO_CORRECT: attached cover letter cover letter
---------------------------------
NO_CORRECT: he male
---------------------------------
NO_CORRECT: https://pah/to/cv.pdf male
---------------------------------
NO_CORRECT: search submit
---------------------------------
NO_CORRECT: search /pah/to/cv.pdf
---------------------------------
ERROR:  <|im_start|>assistant
{"answer":"https://www.linkedin.com/in/sharghi-sharghi-forous
-----------------------

In [21]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():
    
    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))
    


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]<|im_end|>
<|im_start|>user
email<|im_end|>
<|im_start|>assistant
[email]<|im_end|>


In [22]:
$$$

LABEL_INDEX_TO_KEY

SyntaxError: invalid syntax (115010059.py, line 1)

In [88]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel

embdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)

embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2',cache_folder=cache_dir)#.to('cuda')


embeding_model.eval()
embeding_model

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [11]:
data = pd.read_excel('query_element_3.xlsx')
data

,query,element
0,keywords,"<input autocomplete=""off"" autofocus="""" id=""aut..."
1,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""..."
2,Please tick to confirm that you consent to us ...,"<a href=""/privacy-policy"" target=""_blank"" titl..."
3,which includes details on how you can withdraw...,"<input name=""gdpr_verification"" required="""" ty..."
4,Subscribe,"<button type=""submit"">Subscribe</button>"
...,...,...
1586,Please identify your race Please selectAmerica...,"<select aria-required=""false"" id=""job_applicat..."
1587,Race & Ethnicity Definitions,"<a href=""https://boards.cdn.greenhouse.io/docs..."
1588,Veteran Status Please selectI am not a protect...,"<select aria-required=""false"" id=""job_applicat..."
1589,"Completing this form is voluntary, and we hope...","<a href=""https://www.dol.gov/ofccp"" target=""_b..."


In [93]:
embeding_model = get_peft_model(embeding_model,LoraConfig(target_modules=['query','value']))
embeding_model.print_trainable_parameters()


trainable params: 147,456 || all params: 33,507,456 || trainable%: 0.4401


In [ ]:

#c = train_dataset['labels']=='relocate'
#train_dataset.loc[c,'labels'] = 'willing to relocate'
#train_dataset[c]


In [ ]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [54]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [21]:
data.loc[:,'label']=1
train_dataset = datasets.Dataset.from_pandas(data)
train_dataset

Dataset({
    features: ['query', 'element', 'label'],
    num_rows: 1591
})

In [84]:
def batch_tok(example):
    example['query']= embdeding_tokenizer(example['query'])
    example['element']= embdeding_tokenizer(example['element'])
     
train_dataset.map(batch_tok)

Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

TypeError: string indices must be integers, not 'str'

In [90]:
def batch_tok(example):

    return embdeding_tokenizer(example,padding=True,truncation=True,max_length=512,return_tensors='pt')



LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

query_token = embdeding_tokenizer(data['query'].tolist(),padding=False,truncation=True,max_length=512)
element_token = embdeding_tokenizer(data['element'].tolist(),padding=False,truncation=True,max_length=512)
classes_token = embdeding_tokenizer(LABEL_INDEX_TO_KEY,padding=True,truncation=True,max_length=512,return_tensors='pt')


def myheatmap(x):
    map =[]
    for i in range(len(x)):
        sim = torch.cosine_similarity(x,x[i])
        map.append(sim)
    
    
    plt.figure(figsize=(10,10))
    sns.heatmap(map)
    plt.show()
    
    

with torch.no_grad():
    
    output = embeding_model(**classes_token)
    #class_embdeding = torch.mean(output.last_hidden_state,1)
    class_embdeding = mean_pooling(output,classes_token['attention_mask'])
    class_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    #sim = torch.cosine_similarity(class_embdeding,class_embdeding[0])
    
    #print(sim.shape)
    pass
    
i = 151
with torch.no_grad():
    x = torch.tensor(element_token['input_ids'][i:i+1])
    #print(x)
    output = embeding_model(input_ids= x)
    #query_embdeding = torch.mean(output.last_hidden_state,1)
    query_embdeding = mean_pooling(output,torch.tensor(element_token['attention_mask'][i:i+1]))
    #query_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    
    sim = torch.cosine_similarity(class_embdeding,query_embdeding)
    print(pd.DataFrame({'value':sim,'label':LABEL_INDEX_TO_KEY}).sort_values('value',ascending=False)[:5])
    print(LABEL_INDEX_TO_KEY[ sim.argmax() ], embdeding_tokenizer.decode(token_ids=x[0]))
    print(data['element'][i] )
    

#myheatmap(class_embdeding)


TypeError: Sequential.forward() got an unexpected keyword argument 'input_ids'

In [74]:
embeding_model.tokenizer = embdeding_tokenizer
embeding_model.tokenize = embdeding_tokenizer.tokenize

from sentence_transformers import SentenceTransformerModelCardData
from sentence_transformers.losses import CosineSimilarityLoss
#embeding_model.model_card_data = SentenceTransformerModelCardData(train_datasets=train_dataset)

In [94]:
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss=CosineSimilarityLoss(embeding_model),
                           args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=1,
                               per_device_train_batch_size=1
                           )
                           
                           )


trainer.train()

Step,Training Loss
500,0.000300
1000,0.000100
1500,0.000100


TrainOutput(global_step=1591, training_loss=0.0001403093279475764, metrics={'train_runtime': 115.0591, 'train_samples_per_second': 13.828, 'train_steps_per_second': 13.828, 'total_flos': 0.0, 'train_loss': 0.0001403093279475764, 'epoch': 1.0})

In [97]:
test_dataset

,query,element,label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number
...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit


In [127]:
def compute_metric():
    embeding_model.eval()
    acc=0
    
    with torch.no_grad():
        
        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)
        
        for i,x in test_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']
            
            o= embeding_model.encode( x['element'])
            o = torch.tensor(o)
            sim = torch.cosine_similarity(l,o)
            #print(sim.shape)
            #sim = torch.mean(sim,1)
            if LABEL_INDEX_TO_KEY[sim.argmax()]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])
            
           
             
    
    acc/=len(test_dataset)
    print("acc:",acc)
    
    
    
compute_metric()

torch.Size([51, 384])
acc: 0.05


In [ ]:
$$$

In [7]:
with open('./html/1.html','r',encoding='utf-8') as f:
    html = f.read()


html = BeautifulSoup(html,'html.parser')

In [63]:
def preprocess_form(form):
    if isinstance(form,str):
        form = BeautifulSoup(form,'html.parser')
        
    form:PageElement
    comments=form.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    all_tags = form.find_all()

    # Remove all tags except for 'link', 'input', and 'select' tags
    i=0
    for element in all_tags:
        element:PageElement
        for k in ['style','class','bounding_box_rect','is_clickable']:
            if k in element.attrs:
                    del element[k]
                    pass
           
        if 'type' in element.attrs and element.attrs['type']=='hidden':
            element.unwrap()
            pass
            
        
        #if element.name  in ['div','text']: 
        if element.name not in ['a', 'input', 'select','radio','button','textarea','checkbox','option']:
            
            element.unwrap()
            #print(element)
            pass
        else:
            #element['backend_node_id']=i
            pass

            
                
            i+=1
            
    #print(form.prettify() )
    return form
    
import re

def get_query_element(f):
    query_element=[]
    mytext  = ''
    texts = []
    
    for element in f:
        # Get the text before the element
        #print('-----------------------------')
        if element.name == None:
            t = element.text.replace('\n','').replace('\t','').strip()
            if t!='' and len(t)>2:
                #mytext+=t+'\n '
                texts.append(t)
        else:
            mytext = '\n'.join(texts[-1:])
            query_element.append({'query':mytext,'element': str(element).replace('\n','').replace('  ','').strip()})
            mytext =''
            texts=[]
        
    return query_element


def get_query_html(f):
    f:PageElement
    childs = f.find_all(string=True,recursive=False)
    for c in childs:
        print('---------------------')
        print(c)
    pass
    
    
    
print(forms.loc[10,'form'])
f = preprocess_form(forms.loc[25,'form'])
d = get_query_element(f)
d = pd.DataFrame(d)
for i,x in d.iterrows():
    print(x['query'])
    print(x['element'])
    print('--------------------------------------------------')

<form submit="portal-eeo-survey =&gt; submit(event)">
 <rec-form-component cx-prop-direction="vertical" cx-prop-form="create" cx-prop-form-class="rec-form-container career-website-border-style" cx-prop-mandatory-type="star" cx-prop-section="{{sections}}" cx-prop-zcqa="eeo">
 </rec-form-component>
 <div class="cw-submit text-center">
  <lyte-button click="portal-eeo-survey =&gt; cancel()" cx-prop-zcqa="eeo-cancel" id="apply_button" lt-prop-color="{{brand_color}}">
   <template is="registerYield" yield-name="text">
   </template>
  </lyte-button>
  <lyte-button click="portal-eeo-survey =&gt; submit(event)" cx-prop-zcqa="eeo-submit" lt-prop-background-color="{{cxPropBrandColor}}" lt-prop-type="primary">
   <template is="registerYield" yield-name="text">
   </template>
  </lyte-button>
 </div>
</form>


<input aria-label="Eingabe Email" autocomplete="new-email" data-cy="formInputEmail" data-validation-type="emailValidation" name="email" placeholder="E-Mail-Adresse" type="email"/>
---------

In [25]:
forms=pd.read_excel('forms copy.xlsx',index_col=0).reset_index(drop=True)

In [57]:
for _ ,f in forms.iterrows():
    print(_)
    #print(f['form'])
    f = str(f['form'])
    #print(f)
    #f = BeautifulSoup(str(f),'html.parser')
    f = preprocess_form(f)
    #f = f.find_all()
    #print(f)
    d = get_query_element(f)
    d = pd.DataFrame(d)
    #print(d)
    for i,x in d.iterrows():
        print(x['query'])
        print(x['element'])
        print('--------------------------------------------------')
        
        
    print('***************************************************')
    input()

0
Browse CV to autocomplete your application. You can still fill your profile manually.
<input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>
--------------------------------------------------
***************************************************
1
Name *
<input name="first_name" placeholder="First" required="" type="text"/>
--------------------------------------------------
This field contains invalid characters
<input name="last_name" placeholder="Last" required="" type="text"/>
--------------------------------------------------
Email *
<input name="email" placeholder="yourmail@domain.com" required="" type="email"/>
--------------------------------------------------
Phone
<input name="phone" placeholder="+49 176 123 4455" type="text"/>
--------------------------------------------------
Available from *
<input name="available_from" placeholder="" required="" type="text"/>
--------------------------------------------------
GitHub
<input name="custom_attribute_529055"

KeyboardInterrupt: Interrupted by user